In [1]:
from bs4 import BeautifulSoup
from splinter import Browser
import pandas as pd
from datetime import datetime, timedelta
from pprint import pprint
import time
from dotenv import load_dotenv
load_dotenv()
import os

In [2]:
user = os.getenv('USER')
password = os.getenv('password')

In [8]:
#Create the browser
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

try:

    #Visit the website in 'url'
    url = 'https://www.appannie.com/dashboard/home'
    browser.visit(url)

    #Enter login info
    time.sleep(1)
    browser.find_by_name(name = 'username').type(user)
    browser.find_by_css('input')[1].type(password)
    browser.find_by_tag('button').click()

    #########
    def checker(func,go=None):
        #func is a 'is_present' boolean
        for i in range(3):
            if func:
                go
                break
            else:
                browser.reload()
                time.sleep(2)
    ##########

    #Check for presence of 'Top Chart' tab, click tab
    checker((browser.is_element_present_by_text('Top Charts',wait_time=2)),browser.find_by_text('Top Charts').click())

    #Create date range
    today = datetime.now()
    dates = [(today - timedelta(days=i)).strftime("%Y-%m-%d") for i in range(90)]



    #https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date=2020-12-01&feed=All&page_number=0&page_size=500&rank_sorting_type=rank
    #list of dictionaries, each one a different day
    list_of_builds = []

    for moment in dates:
        #get top chart info
        url = f'https://www.appannie.com/apps/ios/top-chart?country=US&category=36&device=iphone&date={moment}&feed=All&page_number=0&page_size=500&rank_sorting_type=rank'
        browser.visit(url)
        time.sleep(5)
        html = browser.html
        soup = BeautifulSoup(html,'html.parser')

        checker(browser.is_element_present_by_css('table'))

        all_trs = soup.table.tbody.find_all('tr')
        build_df = {'Rank':[],'Date': moment,'Free':[],'Paid':[],'Grossing':[]}
        for _ in all_trs:
            current_rank_apps = [elem.text for elem in _.select('a span')]

            rank_num = _.span.text
            build_df['Rank'].append((rank_num))
            build_df['Free'].append([current_rank_apps[0],current_rank_apps[1]])
            build_df['Paid'].append([current_rank_apps[2],current_rank_apps[3]])
            build_df['Grossing'].append([current_rank_apps[4],current_rank_apps[5]])

        list_of_builds.append(build_df)

    #concatenate all dataframes
    builds_to_concat = [pd.DataFrame(_).set_index('Date') for _ in list_of_builds]
    sep3_dec1_t500 = pd.concat(builds_to_concat)
except:
    browser.quit()
finally:
    browser.quit()

In [13]:
sep3_dec1_t500

,Rank,Free,Paid,Grossing
Date,,,,
2020-12-01,1,"[Roof Rails, Voodoo]","[Minecraft, Mojang]","[YouTube: Watch, Listen, Stream, Google]"
2020-12-01,2,"[ZOOM Cloud Meetings, Zoom Video Communications]","[Bloons TD 6, Ninja Kiwi]","[Clash of Clans, Supercell]"
2020-12-01,3,"[YouTube: Watch, Listen, Stream, Google]","[Procreate Pocket, Savage Interactive]","[Disney+, Disney]"
2020-12-01,4,"[Among Us!, InnerSloth]","[Monopoly, Marmalade]","[Tinder - Dating & Make Friends, Match Group]"
2020-12-01,5,"[TikTok, TikTok Pte Ltd]","[Heads Up! - Trivia on the go, Warner Bros]","[Roblox, Roblox]"
...,...,...,...,...
2020-09-03,496,"[Bike Race: Free Style Games, Wildlife Studios]","[Boris and the Dark Survival, Joey Drew]","[Wattpad - Read & Write Stories, Wattpad]"
2020-09-03,497,"[Geometry Dash Lite, RobTop]","[Monument Valley 2, ustwo]","[NFL, NFL Enterprises]"
2020-09-03,498,"[Southwest Airlines, Southwest Airlines]","[Minitube for Youtube, ASN Group]","[Lingokids - playlearning™, Lingokids]"
